In [1]:
# for automatic reloading of libraries
%load_ext autoreload
%autoreload 2
import datetime
import time

import tensorflow as tf
import numpy as np

import datasets
from model import Network

/home/jirka/bakalarka/jupyter-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
MUSICNET_ROOT = "/home/jirka/bakalarka/melody_extraction/datasets/musicnet"
test_data = datasets.musicnet_dataset(MUSICNET_ROOT, "test")
#test_subset = ["musicnet_test_2303", "musicnet_test_2382", "musicnet_test_1819"]
#train_data1 = [aa for aa in test_data if aa.audio.uid not in test_subset]
#test_data = [aa for aa in test_data if aa.audio.uid in test_subset]
train_data = datasets.musicnet_dataset(MUSICNET_ROOT, "train")
#train_data = train_data1+train_data2
print("---")
print(len(test_data), len(train_data))

musicnet_test_2556_16000 2.52 min
musicnet_test_2628_16000 2.08 min
musicnet_test_2382_16000 1.97 min
musicnet_test_2191_16000 1.71 min
musicnet_test_2303_16000 1.54 min
musicnet_test_1819_16000 2.96 min
musicnet_test_2416_16000 2.32 min
musicnet_test_2298_16000 2.56 min
musicnet_test_2106_16000 3.78 min
musicnet_test_1759_16000 3.24 min
musicnet_train_2341_16000 7.68 min
musicnet_train_1893_16000 5.43 min
musicnet_train_2462_16000 8.16 min
musicnet_train_2294_16000 3.81 min
musicnet_train_1876_16000 5.94 min
musicnet_train_2244_16000 3.57 min
musicnet_train_2436_16000 11.41 min
musicnet_train_2406_16000 10.20 min
musicnet_train_1742_16000 15.41 min
musicnet_train_1768_16000 4.85 min
musicnet_train_2230_16000 1.41 min
musicnet_train_2310_16000 0.92 min
musicnet_train_2622_16000 8.45 min
musicnet_train_2161_16000 6.62 min
musicnet_train_2618_16000 6.07 min
musicnet_train_2581_16000 5.76 min
musicnet_train_2590_16000 6.51 min
musicnet_train_2241_16000 4.04 min
musicnet_train_1728_16000 4

In [3]:
test_dataset = datasets.AADataset(test_data, 1, 944, shuffle_batches=False)
train_dataset = datasets.AADataset(train_data, 1, 944, shuffle_batches=True)

In [48]:
def create_model(self, args):
    # --- Perceptron baseline
    audio_net = tf.layers.dense(self.window, 500, activation=tf.nn.relu)
    # audio_net = tf.layers.dropout(audio_net, training=self.is_training)
    # audio_net = tf.layers.dense(audio_net, 8192, activation=tf.nn.relu)
    output_layer = tf.layers.dense(audio_net, self.note_range*self.annotations_per_window, activation=None, name="output")
    ref_notes_flat = tf.layers.flatten(self.ref_notes)

    self.note_probabilites = tf.reshape(output_layer, [-1, self.annotations_per_window, self.note_range])
    self.est_notes = tf.cast(tf.greater(self.note_probabilites, 0.5), tf.float32)
    
    self.loss = tf.losses.sigmoid_cross_entropy(ref_notes_flat, output_layer)
    # self.loss = tf.reduce_mean(tf.nn.l2_loss(ref_notes_flat-output_layer)) #+ 1*R

    global_step = tf.train.create_global_step()
    self.training = tf.train.AdamOptimizer().minimize(self.loss, global_step=global_step)
    # self.training = tf.train.GradientDescentOptimizer(.0001).minimize(self.loss, global_step=global_step)

In [49]:
restore = None

# Create logdir name
args = {
    "threads": 6,
    "epochs": 10,
    "batch_size": 64,
    "logdir": None,
    "annotations_per_window": test_dataset.annotations_per_window,
    "window_size": test_dataset.window_size,
    "note_range": 96,
    "samplerate": test_dataset.samplerate
}
name = "{}-bs{}".format(
    datetime.datetime.now().strftime("%m-%d_%H%M%S"),
    args["batch_size"])

if restore:
    name = restore
args["logdir"] = "models/" + name

print(name)
print(args)

# Construct the network
network = Network(threads=args["threads"])
network.construct(args, create_model)

if restore:
    network.restore()

06-24_114913-bs64
{'threads': 6, 'epochs': 10, 'batch_size': 64, 'logdir': 'models/06-24_114913-bs64', 'annotations_per_window': 1, 'window_size': 2048.0, 'note_range': 96, 'samplerate': 16000}


In [50]:
%%time
# Train
train_dataset.reset()
batch_size = args["batch_size"]
b = 0
timer = time.time()
for i in range(args["epochs"]):
    print("=== epoch", i+1, "===")
    while not train_dataset.epoch_finished():
        batch = train_dataset.next_batch(batch_size)
        accuracy, loss = network.train(batch)

        b += 1
        if b % 100 == 0:
            print(".", end="")
        if b % 1000 == 0:
            batch = train_dataset.next_batch(batch_size)
            accuracy, loss = network.train_detailed(batch)
            print("b {0}; t {1:.2f}; acc {2:.2f}; loss {3:.2f}".format(b, time.time() - timer, accuracy, loss))
            timer = time.time()
        if b % 10000 == 0:
            accuracy = network.evaluate(test_dataset, batch_size*2)
            print("epoch {}, batch {}, t {:.2f}, accuracy: {:.2f}".format(i+1, b, time.time() - timer, accuracy))
            timer = time.time()
        if b % 10000 == 0 and b >= 10000:
            network.save()
            print("saving, t {:.2f}".format(time.time()-timer))
            timer = time.time()
print("=== done ===")

=== epoch 1 ===
..........b 1000; t 2.34; acc 0.03; loss 0.11
..........b 2000; t 2.19; acc 0.07; loss 0.11
..........b 3000; t 2.21; acc 0.10; loss 0.11
..........b 4000; t 2.19; acc 0.10; loss 0.10
..........b 5000; t 2.19; acc 0.12; loss 0.11
..........b 6000; t 2.19; acc 0.17; loss 0.10
..........b 7000; t 2.19; acc 0.10; loss 0.11
..........b 8000; t 2.20; acc 0.18; loss 0.10
..........b 9000; t 2.19; acc 0.13; loss 0.09
..........b 10000; t 2.19; acc 0.17; loss 0.09
eval (0.6071440280646536, 0.20976232750833254, 0.18469417522932807, 0.07439123503042763, 0.7158464374612398, 0.061336663523375044, 0.8515743360150425, 0.6746500114750336, 0.2330849849773315, 0.20953244048907024, 0.0510685775614287, 0.7158464374612398, 0.061336663523375044, 0.8282516785460435)
epoch 1, batch 10000, t 25.60, accuracy: 0.18
Model saved in path: models/06-24_114913-bs64/model.ckpt
saving, t 0.20
..........b 11000; t 2.19; acc 0.15; loss 0.09
..........b 12000; t 2.19; acc 0.12; loss 0.10
..........b 13000

KeyboardInterrupt: 

In [46]:
network.save()

Model saved in path: models/06-24_025629-bs64/model.ckpt


In [47]:
network.evaluate(test_dataset, batch_size*2)

eval (0.7008793005226265, 0.4006507463193754, 0.34214720143162886, 0.0990136860616255, 0.5003355676189991, 0.07197571453248493, 0.6713249682131095, 0.7511059371362049, 0.4293623082521316, 0.37588255126730397, 0.0703021241288693, 0.5003355676189991, 0.07197571453248493, 0.6426134062803533)


0.34214720143162886